# Plasma shape optimization

The first step is to build the cost for the optimization problem:

In [1]:
from mayavi import mlab

mlab.init_notebook()

from stellacode.definitions import w7x_plasma
from stellacode.costs import EMCost
from stellacode.surface import IntegrationParams
from stellacode.costs import (
    AggregateCost,
    DistanceCost,
    NegTorCurvatureCost,
    CurrentCtrCost,
)
from stellacode.costs.utils import Constraint

# Cost for matching the
em_cost = EMCost.from_plasma_config(
    plasma_config=w7x_plasma, integration_par=IntegrationParams(num_points_u=16, num_points_v=16), train_currents=False
)

# constraint on a minimum of the plasma coil distance
distance = DistanceCost(
    Sp=em_cost.Sp,
    constraint=Constraint(limit=0.4, distance=0.03, minimum=True, method="quadratic"),
)


# constraint preventing negative toroidal curvature
neg_curv = NegTorCurvatureCost(constraint=Constraint(limit=-0.05, distance=0.1, minimum=True, method="quadratic"))

# Sum of all costs
agg_cost = AggregateCost(costs=[em_cost, distance, neg_curv])


Notebook initialized with ipy backend.


I0000 00:00:1696860273.260017   18057 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Then we can build a coil factory:

In [2]:
from stellacode.surface import WrappedCoil

coil_factory = WrappedCoil.from_plasma(
    surf_plasma=em_cost.Sp,
    surf_type="cylindrical",
    n_harmonics=4,
    factor=6,
)

Finally we create the optimizer:

In [3]:
from stellacode.optimizer import Optimizer

opt = Optimizer.from_cost(
    agg_cost,
    coil_factory,
    method="L-BFGS-B",
    kwargs=dict(
        options={"disp": True, "maxiter": 1},
    ),
)


And now let's run the optimization:

In [4]:
cost, metrics, results, optimized_params = opt.optimize()
print(metrics)

err_max_B: 9.284683227539062, cost_B: 65.13337707519531, em_cost: 104125618454528.0, cost_J: 1041256167768064.0, max_j: 3331410.0, min_distance: 0.06159818544983864, cost_distance: 42.75738525390625, min_v_curvature: 2.2907443046569824, cost_neg_pol_curv: 0.0, total_cost: 104125618454528.0, 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           27     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.04126D+14    |proj g|=  1.12891D+14


 This problem is unconstrained.


err_max_B: 0.00570228835567832, cost_B: 0.06279648840427399, em_cost: 2152646049792.0, cost_J: 21526459973632.0, max_j: 2568600.75, min_distance: 0.06270990520715714, cost_distance: 3.2304534912109375, min_v_curvature: 2.1033287048339844, cost_neg_pol_curv: 0.0, total_cost: 2152646049792.0, 

At iterate    1    f=  2.15265D+12    |proj g|=  1.04859D+14

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   27      1      2      1     0     0   1.049D+14   2.153D+12
  F =   2152646049792.0000     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
{'err_max_B': Array(0.0057022, dtype=float32), 'cost_B': Array(0.062

We can now plot the results (yes the parameters of the coil factory werre changed in place during the optimization):

In [5]:
coil_surface = coil_factory().get_coil(results.phi_mn_wnet_cur)

coil_surface.plot(vector_field=coil_surface.j_3d)

We can as well set a limit on the plasma current:

In [6]:
# constraint on the maximal current
current_limit = 100
current_ctr = CurrentCtrCost(constraint=Constraint(limit=current_limit, distance=0.3*current_limit, minimum=False, method="quadratic"))

# Sum of all costs
agg_cost = AggregateCost(costs=[em_cost, distance, neg_curv, current_ctr])

# In this case it is better to set the current parameters as trainable parameters because the optimal current 
# configuration is no more the one given by regcoil.
em_cost = EMCost.from_plasma_config(
    plasma_config=w7x_plasma,
    integration_par=IntegrationParams(num_points_u=16, num_points_v=16),
    train_currents=True
)


opt = Optimizer.from_cost(
    agg_cost,
    coil_factory,
    method="L-BFGS-B",
    kwargs=dict(
        options={"disp": True, "maxiter": 1},
    ),
)


cost, metrics, results, optimized_params = opt.optimize()
print(metrics)

err_max_B: 0.00570228835567832, cost_B: 0.06279648840427399, em_cost: 2152646049792.0, cost_J: 21526459973632.0, max_j: 2568600.75, min_distance: 0.06270990520715714, cost_distance: 3.2304534912109375, min_v_curvature: 2.1033287048339844, cost_neg_pol_curv: 0.0, cost_j_ctr: 0.0, total_cost: 2152646049792.0, 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           27     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.15265D+12    |proj g|=  1.04859D+14


 This problem is unconstrained.


err_max_B: 19.70716094970703, cost_B: 45.277244567871094, em_cost: 101663343902720.0, cost_J: 1016633455804416.0, max_j: 3125486.5, min_distance: 0.047008953988552094, cost_distance: 34.470062255859375, min_v_curvature: 2.468585968017578, cost_neg_pol_curv: 0.0, cost_j_ctr: 0.0, total_cost: 101663343902720.0, 
err_max_B: 3.523611307144165, cost_B: 9.389375686645508, em_cost: 13038166999040.0, cost_J: 130381667893248.0, max_j: 2674650.75, min_distance: 0.048347875475883484, cost_distance: 19.400501251220703, min_v_curvature: 2.1502301692962646, cost_neg_pol_curv: 0.0, cost_j_ctr: 0.0, total_cost: 13038166999040.0, 
err_max_B: 0.002661012578755617, cost_B: 0.056778840720653534, em_cost: 608826621952.0, cost_J: 6088266088448.0, max_j: 2557953.75, min_distance: 0.0736687034368515, cost_distance: 0.9170469641685486, min_v_curvature: 2.1116793155670166, cost_neg_pol_curv: 0.0, cost_j_ctr: 0.0, total_cost: 608826621952.0, 
err_max_B: 0.0026619723066687584, cost_B: 0.056840553879737854, em_cos

Finally we can first solve with regcoil and use this solution to start an optimization. This can speed up the optimization:

In [11]:
import numpy as onp
coil_factory = WrappedCoil.from_plasma(
    surf_plasma=em_cost.Sp,
    surf_type="toroidal",
    n_harmonics=4,
    factor=6,
)
coil_surface = coil_factory()

em_cost2 = EMCost.from_plasma_config(
    plasma_config=w7x_plasma,
    integration_par=IntegrationParams(num_points_u=16, num_points_v=16),
    train_currents=False
)
metrics_df, results = em_cost2.cost_multiple_lambdas(coil_surface, onp.logspace(-30, 0, 30))


lamb_val = metrics_df.loc[metrics_df["max_j"] < 1e6 * current_limit].index[0]


coil_factory.set_phi_mn(results[lamb_val].phi_mn_wnet_cur[2:])


opt = Optimizer.from_cost(
    agg_cost,
    coil_factory,
    method="L-BFGS-B",
    kwargs=dict(
        options={"disp": True, "maxiter": 1},
    ),
)


cost, metrics, results, optimized_params = opt.optimize()
print(metrics)

err_max_B: 2.1029207706451416, cost_B: 48.317298889160156, em_cost: 104669753901056.0, cost_J: 1046697555787776.0, max_j: 2714388.25, min_distance: 0.032860349863767624, cost_distance: 143.66049194335938, min_v_curvature: 0.7768245935440063, cost_neg_pol_curv: 0.0, cost_j_ctr: 0.0, total_cost: 104669753901056.0, 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           90     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.04670D+14    |proj g|=  1.15109D+14


 This problem is unconstrained.


err_max_B: 0.012706311419606209, cost_B: 0.06874313205480576, em_cost: 4007923810304.0, cost_J: 40079238103040.0, max_j: 2202772.5, min_distance: 0.03834056854248047, cost_distance: 15.531754493713379, min_v_curvature: 0.6009621024131775, cost_neg_pol_curv: 0.0, cost_j_ctr: 0.0, total_cost: 4007923810304.0, 

At iterate    1    f=  4.00792D+12    |proj g|=  1.07848D+14

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   90      1      2      1     0     0   1.078D+14   4.008D+12
  F =   4007923810304.0000     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
{'err_max_B': Array(0.01270631, dtype=float32), 'co